In [1]:
import pybedtools as pyb
import vcf
import pandas as pd

Loading VCF produced by Allelotype (of lobSTR)

In [16]:
VCF_FILE_NAME = '../data/AllGenesAllAccessions.merged.bam.sorted.annotated.vcf'

Loading in metadata from SRA (https://www.ncbi.nlm.nih.gov/Traces/study/?acc=SRP056687) and table at http://1001genomes.org/accessions.html

In [18]:
AccList = pd.read_csv("../data/AccListFrom1001Arabidopsis.csv")
AccList.rename(columns={'tg_ecotypeid': 'Ecotype'}, inplace=True)

SraRunTable = pd.read_csv("../data/SraRunTable.txt", sep = "\t")
SraRunTable.rename(columns={'ecotype_s': 'Ecotype'}, inplace=True)

AccListSraRunTable = pd.merge(AccList, SraRunTable)

Function to make DataFrame of VCF file, excluding samples not called and with inferred genotype.   

In [24]:
def make_DF_from_VCF(df_filename, vcf_filename):
    fh = open(df_filename, 'w')
    tab = '\t'
    newline = '\n'
    vcf_reader = vcf.Reader(open(vcf_filename, 'r'))
    for record in vcf_reader:
        for sample in vcf_reader.samples:
            if record.genotype(sample)['DP'] >= 1:
                if record.genotype(sample)['GT'] != './.':
                    if None not in record.ALT:

                        GT1 = int(record.genotype(sample)['GT'].split('/')[0]) - 1
                        GT2 = int(record.genotype(sample)['GT'].split('/')[1]) - 1

                        ref = record.REF

                        if GT1 == -1:
                            VAR = record.REF
                        else:
                            VAR = record.ALT[GT1]
                        if GT2 == -1:
                            VAR2 = record.REF
                        else:
                            VAR2 = record.ALT[GT2]

                        gene = 'NA'

                        try:
                            gene = record.INFO['GENE']
                        except:
                            pass


                        Depth = record.genotype(sample)['DP']
                        End = str(record.INFO['END'])
                        NS = str(record.INFO['NS'])

                        output = (record.CHROM + tab + tab + gene + tab + str(record.POS) + tab + str(End) + tab + 
                                  sample + tab + str(Depth) + tab +
                                  record.INFO['MOTIF'] + tab + str(len(record.INFO['MOTIF'])) +
                                  tab + ref +
                                  tab + str(VAR) + tab + str(len(VAR)) + tab + str(VAR2) + tab + str(len(VAR2)) 
                                  + newline)
                        fh.write(output)
    fh.close()

Running function. takes ~30 min, produces a 1.6 GB DataFrame

In [25]:
DF_FILE_NAME = '../data/STR_Arabidopsis.dataframe.annotated.tsv'

make_DF_from_VCF(DF_FILE_NAME,VCF_FILE_NAME)

We filter for depth and allele frequency in the 'FilteringDF' notebook.